<a href="https://colab.research.google.com/github/Nacxht/Clean-or-Messy/blob/main/clean_or_messy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Datasets

In [ ]:
!wget --no-check-certificate \
  https://dicoding-academy-assets.sgp1.cdn.digitaloceanspaces.com/184/messy-vs-clean-room.zip \
  -O /tmp/messy_vs_clean_room.zip

--2023-12-05 08:22:45--  https://dicoding-academy-assets.sgp1.cdn.digitaloceanspaces.com/184/messy-vs-clean-room.zip
Resolving dicoding-academy-assets.sgp1.cdn.digitaloceanspaces.com (dicoding-academy-assets.sgp1.cdn.digitaloceanspaces.com)... 104.18.13.192, 104.18.12.192, 2606:4700::6812:cc0, ...
Connecting to dicoding-academy-assets.sgp1.cdn.digitaloceanspaces.com (dicoding-academy-assets.sgp1.cdn.digitaloceanspaces.com)|104.18.13.192|:443... connected.
HTTP request sent, awaiting response... 200 OK
Cookie coming from dicoding-academy-assets.sgp1.cdn.digitaloceanspaces.com attempted to set domain to digitaloceanspaces.com
Length: 70392746 (67M) [application/zip]
Saving to: ‘/tmp/messy_vs_clean_room.zip’

/tmp/messy_vs_clean 100%[===================>]  67.13M   136MB/s    in 0.5s    

2023-12-05 08:22:46 (136 MB/s) - ‘/tmp/messy_vs_clean_room.zip’ saved [70392746/70392746]



# Import Library

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import numpy as np

import zipfile
import os
from google.colab import files

# Mengekstrak Data
Mengekstrak data dari zip yang telah diunduh.
Lalu mendefinisikan nama direktori untuk data latih dan data validasi

In [ ]:
# Ekstrak data dari zip
local_zip = '/tmp/messy_vs_clean_room.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/images'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

# Augmentasi Gambar
proses augmentasi gambar menggunakan image data generator

In [ ]:
# Train datagen
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Test datagen
test_datagen = ImageDataGenerator(
    rescale=1./255
)

# Menyiapkan Data Train & Validasi

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir, # Direktori data latih
    target_size=(150, 150), # Mengubah seluruh resolusi gambar menjadi 150 * 150 pixel
    batch_size=4,

    # Karena klasifikasi 2 kelas, maka menggunakan class_mode='binary'
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir, # Direktori data validasi
    target_size=(150, 150), # Mengubah resolusi gambar menjadi 150 * 150 pixel
    batch_size=4,

    # Karena klasifikasi 2 kelas, maka menggunakan class_mode='binary'
    class_mode='binary'
)

Found 192 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


# Membuat Model CNN

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Summary Dari Arsitektur Model

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 17, 17, 128)       0

# Melakukan Compile Pada Model
Karena ini merupakan klasifikasi biner, maka kita akan menggunakan loss function "binary_crossentropy". \
Optimizer nya menggunakan "Adam optimizer" karena mudah diterapkan.

In [ ]:
model.compile(
    loss='binary_crossentropy',
    optimizer=tf.optimizers.Adam(),
    metrics=['accuracy']
)

# Melatih Model


In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=25, # Berapa banyak batch yang akan dieksekusi pada tiap epoch
    epochs=20, # Tambahkan epoch jika akurasi model belum optimal
    validation_data=validation_generator, # Menampilkan akurasi pengujian data validasi
    validation_steps=5, # Berapa batch yang akan dieksekusi pada setiap epoch
    verbose=2
)

Epoch 1/20
25/25 - 14s - loss: 0.8170 - accuracy: 0.5000 - val_loss: 0.6882 - val_accuracy: 0.5500 - 14s/epoch - 547ms/step
Epoch 2/20
25/25 - 1s - loss: 0.6915 - accuracy: 0.5400 - val_loss: 0.6836 - val_accuracy: 0.7500 - 1s/epoch - 43ms/step
Epoch 3/20
25/25 - 2s - loss: 0.6823 - accuracy: 0.6100 - val_loss: 0.6839 - val_accuracy: 0.5000 - 2s/epoch - 74ms/step
Epoch 4/20
25/25 - 1s - loss: 0.6916 - accuracy: 0.4900 - val_loss: 0.6446 - val_accuracy: 0.5500 - 1s/epoch - 53ms/step
Epoch 5/20
25/25 - 1s - loss: 0.6672 - accuracy: 0.6300 - val_loss: 0.6334 - val_accuracy: 0.7000 - 1s/epoch - 41ms/step
Epoch 6/20
25/25 - 1s - loss: 0.6708 - accuracy: 0.6000 - val_loss: 0.5685 - val_accuracy: 0.8000 - 1s/epoch - 42ms/step
Epoch 7/20
25/25 - 1s - loss: 0.6666 - accuracy: 0.6000 - val_loss: 0.6194 - val_accuracy: 0.7000 - 1s/epoch - 42ms/step
Epoch 8/20
25/25 - 1s - loss: 0.5978 - accuracy: 0.7000 - val_loss: 0.5783 - val_accuracy: 0.6500 - 1s/epoch - 50ms/step
Epoch 9/20
25/25 - 1s - loss:

# Menguji Model
Bagian untuk menguji model dengan data baru atau data yang belum dikenal oleh model. \
Model akan memprediksi foto suatu ruangan dan menentukan apakah ruangan tersebut bersih atau berantakan.
\
\
Run kode dibawah dan upload gambar suatu ruangan yang anda inginkan, dan model akan menentukan bersih atau tidaknya ruangan dari gambar yang anda kirim.

In [ ]:
%matplotlib inline


uploaded = files.upload()

for fn in uploaded.keys():
  # Memprediksi gambar
  path = fn
  img = image.load_img(path, target_size=(150, 150))

  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  images = np.vstack([x])

  classes = model.predict(images, batch_size=10)

  print(fn)
  if classes == 0:
    print('Clean')
  else:
    print('Messy')